In [1]:
import findspark 
findspark.init() 

import pyspark
from delta import *

In [ ]:
# my_packages = ["org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,net.snowflake:snowflake-jdbc:3.13.26,net.snowflake:spark-snowflake_2.12:2.11.1-spark_3.3"]

# builder = pyspark.sql.SparkSession \
#     .builder \
#     .master("spark://spark-master:7077") \
#     .appName("DeltaLake") \
#     .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
#     .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
#     .config("spark.sql.warehouse.dir", "/opt/spark/work-dir/odp_intra_storage/spark/datalake") \
#     .enableHiveSupport()

# spark = configure_spark_with_delta_pip(builder, extra_packages = my_packages).getOrCreate()

# sc = spark.sparkContext
# sc.setLogLevel('ERROR')

In [ ]:
spark = pyspark.sql.SparkSession.builder.master("spark://spark-master:7077").appName("spark_tester").getOrCreate()

sc = spark.sparkContext
sc.setLogLevel('ERROR')

In [3]:
spark.sql("show databases").toPandas()

,namespace
0,default
1,test_db


In [ ]:
spark.sql("create database if not exists test_db").toPandas()

In [ ]:
spark.sql("desc database test_db").toPandas()

In [ ]:
spark.range(10000000 + 1).write.format("delta").mode('overwrite').saveAsTable("test_db.test_table")

In [ ]:
# spark.sql("drop database test_db cascade")

In [4]:
spark.sql("show tables in test_db").toPandas()

,namespace,tableName,isTemporary
0,test_db,test_table,False


In [10]:
spark.sql("desc detail test_db.test_table").toPandas()

/opt/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/opt/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion
0,delta,62f5f4f8-76fd-4955-aed8-771b908492e4,test_db.test_table,None,file:/opt/spark/work-dir/odp_intra_storage/spa...,2023-02-05 21:09:59.785,2023-02-10 15:41:49.171,[],2,40066724,{},1,2


In [8]:
spark.sql("select * from test_db.test_table limit 2").toPandas()

,id
0,5000000
1,5000001


In [ ]:
spark.range(10000 + 1).write.format("parquet").mode('overwrite').saveAsTable("default.test_table")

In [7]:
spark.sql("select * from default.test_table limit 2").toPandas()

,id
0,5000
1,5001
